# Changing an environment registered in Sinergym

As discussed above, Sinergym has a list of available environments which we can call with `gym.make(<environment_id>)` as long as the Sinergym package is imported into the python script.

In [1]:
import gymnasium as gym
import numpy as np

import sinergym
env = gym.make('Eplus-5zone-hot-continuous-stochastic-v1')

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-stochastic-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-continuous-stochastic-v1 created successfully.


These environment ID's have a number of components defined, not only the building design (*epJSON*), but also the reward function, the action and observation spaces, the definition of variables, meters and actuators, etc.

If you want a new environment, you can define it from scratch in our [environment list](https://github.com/ugr-sail/sinergym/blob/main/sinergym/__init__.py) (take the env demo as example) and run it locally.

Another option (recommended) is starting from one of our environments ID and changing the components you want. You can combine different components obviously.

The way to do that is to add in the `gym.make(<environment_id>)` those parameters of the constructor of the *Sinergym* environment we want to overwrite. 

Many of this updates require the building model changes in order to adapt it to this new features, this will be made by Sinergym automatically. For example, using another weather file requires building location and design days update, using new observation variables requires to update the `Output:Variable` fields, the same occurs with extra configuration context concerned with simulation directly. This new building file version, is saved in the Sinergym output folder, leaving the original intact.

Let's see some things we can update starting from any environment:

## Adding a new reward

As mentioned above, simply add the appropriate parameters to `gym.make()` after specifying the environment ID.

In [2]:
from sinergym.utils.rewards import ExpReward

env = gym.make('Eplus-5zone-hot-continuous-v1', reward=ExpReward, reward_kwargs={
                                                                    'temperature_variable': 'air_temperature',
                                                                    'energy_variable': 'HVAC_electricity_demand_rate',
                                                                    'range_comfort_winter': (20.0, 23.5),
                                                                    'range_comfort_summer': (23.0, 26.0),
                                                                    'energy_weight': 0.1})

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-hot-continuous-v1 created successfully.


You have to specify the **reward** class you are going to use in the environment. A reward function class has several parameters that can be specified by user like temperature variables, weights, etc. Depending on the reward function you are using. In order to be able to define it, we have **reward_kwargs** parameter. See [reward documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/rewards.html) for more information about reward classes and how to create a new one. 

## Updating the weather used in environment

It is possible to substitute the weather file (and the weather variability if you want a stochastic environment) used in the environment by a new one. The methodology is the same than the reward example:

In [3]:
env = gym.make('Eplus-5zone-cool-continuous-stochastic-v1', 
                weather_files='ESP_Granada.084190_SWEC.epw',
                weather_variability=(1.0,0.0,0.001))

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-cool-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-cool-continuous-stochastic-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-cool-continuous-stochastic-v1 created successfully.


It is possible to **add more than one weather** file too. In that case, Sinergym will select one of the available weathers randomly and will adapt the building model to that location in each episode:

In [4]:
env = gym.make('Eplus-5zone-cool-continuous-stochastic-v1', 
                weather_files=['ESP_Granada.084190_SWEC.epw','FIN_Helsinki.029740_IWEC.epw','PRT_Lisboa.085360_INETI.epw'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-cool-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-cool-continuous-stochastic-v1-res2]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment 5zone-cool-continuous-stochastic-v1 created successfully.


## Changing observation and action spaces

By default, the IDs of the predefined environments in *Sinergym* already have the **time variables**, **variables**, **meters** and **actuators** set up, together observation (time variables, variables and meters) and action (actuators) spaces for those components.

However, it can be overwritten by a new definition of them. On the one hand, we will have to define the name of the 
**time variables**, **variables**, **meters** or **actuators**, and on the other hand, the definition of the **action space** (and an **action mapping** if it is a discrete environment). Observation space is calculated automatically by Sinergym. It is possible to change a subset of these components too. For more information about the structure definitions shown bellow, visit the documentation.

In [5]:
import gymnasium as gym
import numpy as np

import sinergym

new_time_variables=['month', 'day_of_month', 'hour']

new_variables={
    'outdoor_temperature': ('Site Outdoor Air Drybulb Temperature', 'Environment'),
    'outdoor_humidity': ('Site Outdoor Air Relative Humidity', 'Environment'),
    'wind_speed': ('Site Wind Speed', 'Environment'),
    'wind_direction': ('Site Wind Direction', 'Environment'),
    'diffuse_solar_radiation': ('Site Diffuse Solar Radiation Rate per Area', 'Environment'),
    'direct_solar_radiation': ('Site Direct Solar Radiation Rate per Area', 'Environment'),
    'west_htg_setpoint': ('Zone Thermostat Heating Setpoint Temperature', 'West Zone'),
    'east_htg_setpoint': ('Zone Thermostat Heating Setpoint Temperature', 'East Zone'),
    'west_clg_setpoint': ('Zone Thermostat Cooling Setpoint Temperature', 'West Zone'),
    'east_clg_setpoint': ('Zone Thermostat Cooling Setpoint Temperature', 'East Zone'),
    'west_zone_temperature': ('Zone Air Temperature', 'West Zone'),
    'east_zone_temperature': ('Zone Air Temperature', 'East Zone'),
    'west_zone_air_humidity': ('Zone Air Relative Humidity', 'West Zone'),
    'east_zone_air_humidity': ('Zone Air Relative Humidity', 'East Zone'),
    'HVAC_electricity_demand_rate': ('Facility Total HVAC Electricity Demand Rate', 'Whole Building')
}

new_meters={
    'east_zone_electricity':'Electricity:Zone:EAST ZONE',
    'west_zone_electricity':'Electricity:Zone:WEST ZONE',
}

new_actuators = {
    'Heating_Setpoint_RL': (
        'Schedule:Compact',
        'Schedule Value',
        'Heating Setpoints'),
    'Cooling_Setpoint_RL': (
        'Schedule:Compact',
        'Schedule Value',
        'Cooling Setpoints')
}

new_action_mapping = {
    0: (15, 30),
    1: (16, 29),
    2: (17, 28),
    3: (18, 27),
    4: (19, 26),
    5: (20, 25),
    6: (21, 24),
    7: (22, 23),
    8: (22, 22),
    9: (21, 21)
}

new_action_space = gym.spaces.Discrete(10)

env = gym.make('Eplus-datacenter-cool-discrete-stochastic-v1', 
                time_variables=new_time_variables,
                variables=new_variables,
                meters=new_meters,
                actuators=new_actuators,
                action_space=new_action_space,
                action_mapping=new_action_mapping
            )

print('New environment observation varibles (time variables + variables + meters): {}'.format(env.observation_variables))
print('New environment action varibles (actuators): {}'.format(env.action_variables))
for i in range(1):
    obs, info = env.reset()
    rewards = []
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        rewards.append(reward)
    print(
        'Episode ',
        i,
        'Mean reward: ',
        np.mean(rewards),
        'Cumulative reward: ',
        sum(rewards))
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [datacenter-cool-discrete-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-datacenter-cool-discrete-stochastic-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment datacenter-cool-discrete-stochastic-v1 created successfully.
New environment observation varibles (time variables + variables + meters): 

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-datacenter-cool-discrete-stochastic-v1-res1/Eplus-env-sub_run1/USA_WA_Port.Angeles-William.R.Fairchild.Intl.AP.727885_TMY3_Random_1.0_0.0_0.001.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-datacenter-cool-discrete-stochastic-v1-res1/Eplus-env-sub_run1/output', '/workspaces/sinergym/examples/Eplus-env-datacenter-cool-discrete-stochastic-v1-res1/Eplus-env-sub_run1/2ZoneDataCenterHVAC_wEconomizer.epJSON']
[ENVIRONMENT] (INFO) : Episode 1 started.
[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%
Episode  0 Mean reward:  -1.0457358226428322 Cumulative reward:  -36642.58322540486
[ENVIRONMENT] (INFO) : Environment closed. [datacenter-cool-discrete-stochastic-v1]


In case the definition has some inconsistency, such as the spaces do not fit with the variables, the 
observation variables do not exist, etc. *Sinergym* will display an error.

The time variables, variables, meters or actuators definition must be correct for Energyplus Engine.
Otherwise, an error message will be shown by Sinergym. See documentation for more information.


## Adding more extra configuration

It is possible to update any argument of the environment constructor in Sinergym. Consequently, we can make other changes such as the name of the environment or the maximum number of episodes it will store in its output or the normalization in the gym actions interface.

In [6]:
env = gym.make('Eplus-datacenter-cool-continuous-stochastic-v1', 
                env_name='new_env_name',
                flag_normalization=False,
                max_ep_data_store_num=20
                )

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [new_env_name]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-new_env_name-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment new_env_name created successfully.


You can even add a dictionary with extra parameters that update building model with some new context concerned with simulation directly:

In [7]:
extra_conf={
    'timesteps_per_hour':6,
    'runperiod':(1,1,1991,2,1,1991),
}

env = gym.make('Eplus-datacenter-cool-continuous-stochastic-v1', 
                config_params=extra_conf
                )

env.reset()
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [datacenter-cool-continuous-stochastic-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-datacenter-cool-continuous-stochastic-v1-res1]
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 1, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35040
[MODELING] (INFO) : Model Config is correct.
[ENVIRONMENT] (INFO) : Environment datacenter-cool-continuous-stochastic-v1 created successfully.
#---------------------------------------------------------------------

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[SIMULATOR] (INFO) : Handles initialized.
[SIMULATOR] (INFO) : Handles are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |**************************************************-------------------------------------------------| 50%
[ENVIRONMENT] (INFO) : Environment closed. [datacenter-cool-continuous-stochastic-v1]


For more information about extra configuration parameters, see our [Extra configuration documentation](https://ugr-sail.github.io/sinergym/compilation/html/pages/extra-configuration.html)